# WorldCereal Embeddings Demo

This notebook demonstrates how to generate spatio-temporal WorldCereal embeddings through the openEO backend and visualize the resulting multiband GeoTIFF. The pretrained geospatial model under the hood is NASA Harvest's Presto model, finetuned on WorldCereal reference data.

**Workflow overview**:
1. Configure parameters for the embeddings generation.
2. Interactively select an Area of Interest (AOI) and processing date range.
3. Build and run the openEO process graph to create an asynchronous job.
4. Download the resulting embeddings GeoTIFF.
5. Inspect and visualize the embeddings (band statistics, PCA projection, pseudo-color rendering).

Use this as a starting point to explore model outputs or integrate embeddings into downstream ML tasks.

> Tip: Keep your AOI modest in size initially to reduce processing time. Expand once the workflow is validated.

> Note: Maximum area for this processing is limited as larger areas may run into computational issues. If you want to run this at scale, openEO's job manager will be required.

In [ ]:
# Core imports for building the WorldCereal embeddings process graph
from worldcereal.job import create_embeddings_process_graph
from worldcereal.parameters import EmbeddingsParameters

# Utility/visualization imports
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Raster I/O for visualization after download
import rasterio

OUTPUT_PATH = Path("worldcereal_embeddings.tif")

## 1. Configure Embedding Parameters

Adjust any parameters below to control the embedding generation. You can override (for example) the model URL or other processing options if exposed by the API. After instantiation we print the effective configuration for transparency.

In [ ]:
# Instantiate embedding parameters (override defaults by passing arguments)
# Example: EmbeddingsParameters(presto_model_url="s3://bucket/custom_model.onnx")
embedding_params = EmbeddingsParameters()

# Display the resolved parameters (assumes dataclass / attrs-like repr)
print("Embedding Parameters:\n", embedding_params)

## 2. Select Area of Interest & Time Range

Use the interactive map below to draw/select your AOI. Then use the date slider to choose the processing period. Keep ranges short at first to speed up processing.

In [ ]:
from worldcereal.utils.map import ui_map

# Launch interactive map (draw/select AOI)
map = ui_map(area_limit=400)  # To keep processing as a demo manageable
map

A key feature of WorldCereal is its dynamic nature to compute embeddings (and infer downstream classes) on dynamically chosen temporal windows, aligned to the seasonality of interest. We compute the embeddings based on a one-year time range that can be chosen below.

In [ ]:
from notebook_utils.dateslider import date_slider

# Create interactive date range selector widget
slider = date_slider()
slider

### Capture Selected Extent & Period
Run the cell below after finalizing the AOI and dates to capture them for the openEO job.

In [ ]:
# Retrieve AOI extent and date range
extent = map.get_extent()
processing_period = slider.get_selected_dates()

print("Extent (bbox or geometry):", extent)
print("Processing period:", processing_period)

## 3. Run openEO Embeddings Job
This step builds the embeddings process graph, submits it as a job, waits for completion, and downloads the result as a multiband GeoTIFF.

In [ ]:
from worldcereal.job import INFERENCE_JOB_OPTIONS

scale_uint16 = True  # If False, the original Float32 values are returned

# Create WorldCereal Embeddings openEO process graph
print("Building process graph...")
inference_result = create_embeddings_process_graph(
    spatial_extent=extent,
    temporal_extent=processing_period,
    embeddings_parameters=embedding_params,
    scale_uint16=scale_uint16,
)

# Create the job
print("Creating job...")
job = inference_result.create_job(
    title="WorldCereal Embeddings Map",
    job_options=INFERENCE_JOB_OPTIONS,
)

# Start and wait
print("Starting job (this may take a while depending on AOI & cluster load)...")
job.start_and_wait()
print("Job finished. Retrieving results...")

# Download first GeoTIFF asset
for asset in job.get_results().get_assets():
    if asset.metadata.get("type", "").startswith("image/tiff"):
        print(f"Downloading asset to {OUTPUT_PATH} ...")
        asset.download(str(OUTPUT_PATH))
        break
else:
    raise RuntimeError("No GeoTIFF asset found in job results.")

if not OUTPUT_PATH.exists():
    raise FileNotFoundError(f"Expected output not found: {OUTPUT_PATH}")
print("Download complete.")

## 4. Inspect & Visualize Embeddings
We now load the downloaded multiband embeddings GeoTIFF and produce:
1. Per-band value distribution (histogram)
2. PCA projection (first 3 components) rendered as an RGB image
3. Optional pseudo-color rendering from selected bands

Note: If data were scaled to UInt16, we scale back first to the original Float32 values.

In [ ]:
# Open the embeddings raster and inspect basic metadata
with rasterio.open(OUTPUT_PATH) as ds:
    data = ds.read()  # shape: (bands, height, width)
    profile = ds.profile

# Go back to original scale if needed
if scale_uint16:
    # Empirically determined scaling factors for Presto-based WorldCereal embeddings
    offset = -6
    scale = 0.0002
    data = data.astype(np.float32) * scale + offset

print("Raster profile:\n", profile)
print("Data shape (bands, height, width):", data.shape)

In [ ]:
# Plot histograms for first few embeddings
bands_to_plot = min(6, data.shape[0])
fig, axes = plt.subplots(2, (bands_to_plot+1)//2, figsize=(12, 6))
axes = axes.flatten()
for i in range(bands_to_plot):
    arr = data[i].ravel()
    sample = arr if arr.size < 300_000 else np.random.choice(arr, 300_000, replace=False)
    axes[i].hist(sample, bins=50, color='steelblue', alpha=0.8)
    axes[i].set_title(f'Embedding dimension {i+1}')
plt.tight_layout()
plt.show()

In [ ]:
# Incremental / chunked PCA to avoid memory blow-ups
import numpy as np
from sklearn.decomposition import IncrementalPCA

bands, h, w = data.shape
flat = data.reshape(bands, -1).T  # (pixels, bands)

# Configuration
n_components = 3
fit_sample_max = 400_000   # subset used to fit
chunk_size = 50_000        # number of pixels processed per partial_fit / transform

n_pixels = flat.shape[0]
idx_all = np.arange(n_pixels)

# Select subset for fitting (random) to speed up & reduce RAM
if n_pixels > fit_sample_max:
    fit_idx = np.random.choice(n_pixels, fit_sample_max, replace=False)
else:
    fit_idx = idx_all

ipca = IncrementalPCA(n_components=n_components)

# Pass 1: partial_fit on chunks of the fit subset
for start in range(0, fit_idx.size, chunk_size):
    end = start + chunk_size
    batch_idx = fit_idx[start:end]
    ipca.partial_fit(flat[batch_idx])

print('Explained variance ratio (incremental):', ipca.explained_variance_ratio_)

In [ ]:
# Pass 2: transform entire image in chunks (streaming) and track per-component min/max
components_full = np.empty((n_pixels, ipca.n_components), dtype=np.float32)
comp_min = np.full(ipca.n_components, np.inf, dtype=np.float64)
comp_max = np.full(ipca.n_components, -np.inf, dtype=np.float64)

for start in range(0, n_pixels, chunk_size):
    end = min(start + chunk_size, n_pixels)
    transformed = ipca.transform(flat[start:end])
    components_full[start:end] = transformed
    # Update running min/max
    comp_min = np.minimum(comp_min, transformed.min(axis=0))
    comp_max = np.maximum(comp_max, transformed.max(axis=0))

# Reshape to (h, w, 3)
comp_img = components_full.reshape(h, w, ipca.n_components)

# Normalize using collected min/max (avoid second full pass)
rng = (comp_max - comp_min)
rng[rng == 0] = 1
comp_norm = (comp_img - comp_min) / rng

plt.figure(figsize=(10,10))
plt.imshow(comp_norm[..., :3])
plt.title('Incremental PCA Components 1-3 (as RGB)')
plt.axis('off')
plt.show()

print('Final component ranges:', list(zip(comp_min, comp_max)))

In [ ]:
# Create a pseudo-RGB from three arbitrary embedding bands
pseudo_indices = (0, 1, 2) if data.shape[0] >= 3 else None
if pseudo_indices:
    pseudo = data[list(pseudo_indices), :, :].astype(float)
    # Normalize each selected band individually
    for i in range(3):
        b = pseudo[i]
        pseudo[i] = (b - b.min()) / (b.max() - b.min() + 1e-9)
    rgb = np.transpose(pseudo, (1, 2, 0))
    plt.figure(figsize=(10,10))
    plt.imshow(rgb)
    plt.title(f'Pseudo RGB Bands {pseudo_indices}')
    plt.axis('off')
    plt.show()
else:
    print("Not enough bands for pseudo-RGB composite.")

## 5. Next Steps
You can now:
- Feed embeddings into clustering or dimensionality reduction for pattern discovery.
- Sample embeddings at reference points for model training.
- Compare embeddings across seasons or AOIs.
- ...